## CSE202 - Disign and Analysis of Algorithms - Week 7 - Randomized Algorithms 3: Random Search

日期：2022-11-11

姓名：Yubo Cai

课程：CSE202 - Disign and Analysis of Algorithms 高级算法设计与分析

<img src="https://raw.githubusercontent.com/adimajo/CSE204-2021/master/data/logo.jpg" style="float: left; width: 15%" />

In [1]:
from sympy.matrices import randMatrix
import numpy as np
import matplotlib.pyplot as plt
import timeit
import math
import sympy as sym
import crypt

### 1. Random Walk in a Maze

vertex of `degree` $d(v)=2$ 与一个顶点 $v$关联的边的条数称作该顶点的度 (Degree) (或次数)

#### 1.1 Finite Graph

$n$ vertices $\in \mathbb{N}$, $m$ edges the we got $m \leq \begin{pmatrix}n\\2\end{pmatrix}$

#### 1.2 Adjacency Matrix

$A_{ij} = 1$ if there is an edge from $i$ to $j$ and $A_{ij} = 0$ otherwise

$G$ is `undirected` if $A_{ij} = A_{ji}$ in other words $A(G)$ is symmetric

`Distance` $\Delta (u,v)$ minimal number of edges in a path from $u$ to $v$

#### 1.3 Probabilistic Algorithms

In [ ]:
Input: u initial vertex, v target vertex
While u ≠ v
    Pick a neighbor w of u uniformly at random
    Set u := w
Return

Random variable $X_k$ = vertex visited at $k$ th step ($X_{0} = u$).

Complexity: $T(u, v):=\mathbb{E}\left(\inf \left\{k \geq 1 \mid X_k=v\right\}\right)=? ?$

#### 1.4 Exiting the Maze and Lemma
$$
T(u, v):=\mathbb{E}\left(\inf \left\{k \geq 1 \mid X_k=v, \text { with } X_0=u\right\}\right), \quad p_{u v}= \begin{cases}\frac{1}{d(u)}, & \text { if }(u, v) \in G \\ 0 & \text { otherwise }\end{cases}
$$
这里$d(u)$是节点$u$的度数，$p_{u v}$是从节点$u$到节点$v$的概率

`Lemma 1.`
$$
\sum_{v \mid(u, v) \in G} T(v, u)=2 m-d(u) .
$$
**Proof.**  Decomposing the expectation by the first step of the walk starting at a vertex $w$ gives
$$
T(w, u)=p_{w u}+\sum_{\substack{v \mid(w, v) \in G \\ v \neq u}} \frac{1}{d(w)}(1+T(v, u))
$$
since the first step can lead directly to $u$ with probability $p_{w u}$, or to one of the other vertices $v$ of $w$, from where the expected time to reach $u$ is $T(v, u)$. The expression on the right-hand side simplifies in two ways: the sum over all $d(w)$ neighbors of $w$ (including possibly $u$ ) of $1 / d(w)$ is 1 ; the sum over all $v \neq u$ of the remaining part can be written as the sum over all $v$ minus the term corresponding to $v=u$, which gives
$$
T(w, u)=1+\frac{1}{d(w)} \sum_{v \mid(w, v) \in G} T(v, u)-p_{w u} T(u, u) .
$$
The next step is to multiply this identity by $d(w)$ and sum over all $w \in G$ :
$$
\sum_{w \in G} d(w) T(w, u)=\underbrace{\sum_{w \in G} d(w)}_{2 m}+\underbrace{\sum_{w \in G} \sum_{v \mid(w, v) \in G} T(v, u)}_{\sum_{v \in G} d(v) T(v, u)}-\underbrace{\left(\sum_{w \in G} d(w) p_{w u}\right)}_{d(u)} T(u, u) .
$$
Three simplifications occur in the right-hand side: in the sum of $d(w)$ over all vertices $w \in G$, each edge of the graph is counted exactly twice, so that the first sum is $2 m$; exchanging the order of summation in the second term shows that $T(v, u)$ occurs exactly $d(v)$ times in the sum; in the final sum, $d(w) p_{w u}$ is 1 when $(w, u) \in G$ and 0 otherwise, so that this sum is exactly $d(u)$.

The left-hand side is exactly equal to the middle term of the right-hand side (up to relabeling the index of the sum), so that in the end, we obtain
$$
T(u, u)=\frac{2 m}{d(u)} .
$$
Now, using eq. (1) with $w=u$ and this value of $T(u, u)$ gives
$$
\frac{2 m}{d(u)}=1+\frac{1}{d(u)} \sum_{v \mid(u, v) \in G} T(v, u) .
$$
Multiplying by $d(u)$ concludes the proof.
Each edge $(u, v) \in G$ occurs in a sum such as given by the lemma. As a consequence, $T(u, v) \leq 2 m-1($ since $d(v) \geq 1)$. Next, if $u \rightarrow u_1 \rightarrow \cdots \rightarrow v$ is a path of minimal length $\Delta(u, v)$ from $u$ to $v$, summing these expectations over edges in the path gives the following.

`Proposition 1.` For arbitrary vertices $u$ and $v$ in 
$$G, T(u, v) \leq(2 m-1) \Delta(u, v)$$
`Proposition 2.` The expected time $T(u, \cdot)$ for the walk to visit all nodes starting from $u$ satisfies $$T(u, \cdot) \leq 2 m(n-1)$$

Then we can use Markov's inequality to get the following.
$$
\mathbb{P}(v\ not\ visited\ in\ 4nm\ steps) \leq \frac{1}{4nm}T(u, \cdot) \leq \frac{1}{4nm}2 m(n-1) = \frac{1}{2n} \leq \frac{1}{2}
$$
Apply Monte Carlo method to get the in time $O(mn)$ with memory $O(\log n)$


### 2. Satisfiability Problem 布尔可满足性问题

#### 2.1 Boolean Formulas

Variables $x_1, x_2, \ldots, x_n$ with values in {0,1}={false, ture} and Boolean operators $\neg, \land, \lor$. For example
$$
\begin{aligned}
&F:=\left(x_1 \wedge x_2 \wedge x_3\right) \vee\left(\bar{x}_1 \wedge \bar{x}_2\right) \\
&G:=\left(x_1 \vee \bar{x}_2\right) \wedge\left(\bar{x}_1 \vee x_2\right) \wedge\left(\bar{x}_1 \vee x_3\right) \wedge\left(\bar{x}_2 \vee x_3\right)
\end{aligned}
$$
`Satisfiability 满足性`: existence of an assignment such that $F=1$ or $F=true$
$$
(x_{1}, x_{2}, x_{3})=(0,0,1)\ satisfies\ F
$$
`Clause 析取式`: a disjunction($\lor$) of variables and their negations

`Cube 合取式`: a conjunction($\land$) of variables and their negations

`Conjunctive Normal Form (CNF) 合取范式`: a conjunction($\land$) of clauses. ($G$ is in CNF)

#### 2.2 K-SAT Problem

`SAT Problem`: Given a CNF $F$ with $n$ variables and $m$ clauses, is there an assignment of values to the variables such that $F=1$?

`K-SAT Problem`: Given a CNF where every clause has at most $k$ literals, is there an assignment of values to the variables such that $F=1$? k-SAT问题，其限制formula中每个析取式（称为clause 子句）的size 不超过k，在这种情况下，$m=O(n^k)$  ，这说明，如果我们能找到关于n的多项式时间算法，整个k-SAT问题，对应的时间也是多项式的。

**For $k \geq 3$, no polynomial-time algorithm is known**

#### 2.3 WalkSAT Algorithm 局部随机搜索算法

`Input`: a k-SAT formula $F$ with $n$ variables. `Output`: an assignment of values to the variables such that $F=1$ or FALSE if no such assignment exists.

1. 选择assignment $B \in {0,1}^n$ uniformly at random
2. 重复一下步骤$N$次 
    - 如果formula满足assignment $B$，则返回$B$
    - 选择一个clause $C$ 不满足的
    - 随机选择一个变量$x$在$C$中，使得$B$中$x$的值被改变后，$C$满足
3. 返回FALSE

`实例`
$$
\left(x_1 \vee x_2 \vee x_3\right) \wedge\left(\bar{x}_1 \vee \bar{x}_2 \vee x_3\right) \wedge\left(\bar{x}_1 \vee x_2 \vee \bar{x}_3\right) \wedge\left(x_1 \vee \bar{x}_2 \vee x_3\right)
$$
1. Start with $(0,1,0)$:
$\left(x_1 \vee \bar{x}_2 \vee x_3\right)$ is not satisfied
2. Flip $x_1 \rightarrow(1,1,0)$:
$\left(\bar{x}_1 \vee \bar{x}_2 \vee x_3\right)$ is not satisfied
3. Flip $x_2 \rightarrow(1,0,0)$:
Solved!